In [ ]:
# Gerekli Kütüphanelerin İçe Aktarılması
import os
import random
from shutil import copyfile
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
#Dosya Yolları Belirtiliyor
train_dir = 'DataSet/train/'  # Eğitim verisi dizini
test_dir = 'DataSet/test/'      # Doğrulama verisi dizini

batch_size = 64

In [ ]:
# Eğitim ve Test Verisi İçin Veri Artırma Ve Ön İşleme Fonksiyonu
class TripletDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data_dir, batch_size):
        """
        Triplet loss için veri jeneratörü.

        Args:
        - data_dir (str): Veri dizini (örneğin: "train").
        - batch_size (int): Batch boyutu.
        """
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.classes = os.listdir(data_dir)
        self.classes.sort()
        self.image_paths = {cls: os.listdir(os.path.join(data_dir, cls)) for cls in self.classes}

    def __len__(self):
        return len(self.classes) // self.batch_size

    def __getitem__(self, idx):
        anchors, positives, negatives = [], [], []

        for _ in range(self.batch_size):
            anchor_class = random.choice(self.classes)
            positive_class = anchor_class
            negative_class = random.choice([cls for cls in self.classes if cls != anchor_class])

            anchor_img, positive_img = random.sample(self.image_paths[anchor_class], 2)
            negative_img = random.choice(self.image_paths[negative_class])

            anchor_path = os.path.join(self.data_dir, anchor_class, anchor_img)
            positive_path = os.path.join(self.data_dir, positive_class, positive_img)
            negative_path = os.path.join(self.data_dir, negative_class, negative_img)

            anchors.append(self._load_image(anchor_path))
            positives.append(self._load_image(positive_path))
            negatives.append(self._load_image(negative_path))

        return [np.array(anchors), np.array(positives), np.array(negatives)], None

    def _load_image(self, path):
        img = load_img(path, target_size=(224, 224))  # VGG16 için varsayılan giriş boyutu
        img = img_to_array(img) / 255.0  # Normalizasyon
        return img

In [ ]:
# 3. Model ve Eğitim Süreci
def build_model():
    """
    VGG16 tabanlı transfer learning modeli. Çıkış katmanı 128 boyutlu vektör.
    """
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    x = Flatten()(base_model.output)
    embedding = Dense(128, activation="relu")(x)  # 128 boyutlu vektör
    model = Model(inputs=base_model.input, outputs=embedding)
    return model

def triplet_loss(y_true, y_pred):
    """
    Triplet loss fonksiyonu.
    """
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(pos_dist - neg_dist + 0.2, 0.0)  # Margin = 0.2
    return tf.reduce_mean(loss)

In [ ]:
# Modeli oluştur
model = build_model()
model.compile(optimizer=Adam(learning_rate=0.0001), loss=triplet_loss)

In [ ]:
# Veri jeneratörlerini başlat
train_gen = TripletDataGenerator(train_dir, batch_size=batch_size)
val_gen = TripletDataGenerator(test_dir, batch_size=batch_size)

In [ ]:
# Modeli eğit
model.fit(train_gen, validation_data=val_gen, epochs=100)

In [ ]:
# Modelin Kaydedilmesi
model.save('model.h5')

In [ ]:
# Eğitim ve Doğrulama Doğruluğu Çizimi
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.ylabel('Doğruluk')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Eğitim ve Doğrulama Kaybı Çizimi
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kaybı')
plt.ylabel('Kayıp')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Test Verisinde Tahminler
y_true = val_generator.classes
y_pred_probs = model.predict(val_generator, steps=val_generator.samples // batch_size)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
# Karmaşıklık Matrisi
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Tahmin Edilen Sınıf')
plt.ylabel('Gerçek Sınıf')
plt.title('Karmaşıklık Matrisi')
plt.show()